In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar, minimize
import math

# The Kelly Criterion

In [2]:
def expected_log_growth_rate(f, p, x1, x2):
    """
    Compute the expected logarithmic growth rate given a fraction f invested in a risky asset.

    :param f: Fraction of wealth invested in the risky asset
    :param p: Probability that the asset pays x1 for every dollar wagered
    :param x1: Payoff for every dollar wagered with probability p
    :param x2: Payoff for every dollar wagered with probability (1-p)
    :return: Expected logarithmic growth rate
    """
    return p * np.log(1 + f * (x1 - 1)) + (1 - p) * np.log(1 + f * (x2 - 1))

In [3]:
# Example usage
p = 0.5  # Probability that the asset pays x1 for every dollar wagered
x1 = 3  # Payoff for every dollar wagered with probability p
x2 = .1
# Payoff for every dollar wagered with probability (1-p)

# Investment fractions to evaluate
fractions = [0.02, 0.15, 0.5, 1]

# Compute the expected logarithmic growth rates for each fraction
growth_rates = [expected_log_growth_rate(f, p, x1, x2) for f in fractions]

# Create a pandas DataFrame to display the results in a table
data = {'Fraction': fractions, 'Expected Logarithmic Growth Rate': growth_rates}
table = pd.DataFrame(data)

# Display the table
print(table)

   Fraction  Expected Logarithmic Growth Rate
0      0.02                          0.010528
1      0.15                          0.058669
2      0.50                          0.047655
3      1.00                         -0.601986


In [4]:
def find_optimal_fraction(p, x1, x2):
    """
    Find the optimal fraction of wealth to invest in a risky asset using an optimization package.


    :param p: Probability that the asset pays x1 for every dollar wagered
    :param x1: Payoff for every dollar wagered with probability p
    :param x2: Payoff for every dollar wagered with probability (1-p)
    :return: Optimal fraction of wealth to invest in the risky asset
    """
    result = minimize_scalar(lambda f: -expected_log_growth_rate(f, p, x1, x2), bounds=(0, 1), method='bounded')
    return result.x

In [5]:
# Solve the example
optimal_fraction = find_optimal_fraction(p, x1, x2)
print(f"Optimal fraction of wealth to invest in the risky asset: {optimal_fraction:.4f}")
print(f"Optimal expected logarithmic growth rate: {expected_log_growth_rate(optimal_fraction, p, x1, x2):.4f}")

Optimal fraction of wealth to invest in the risky asset: 0.3056
Optimal expected logarithmic growth rate: 0.0777


# Some Mathematics of Gambling

In [6]:
def find_optimal_fraction_hot_cold(p_hot, p_cold, x1_win, x1_lose, x2_win, x2_lose):
    """
    Find the optimal betting fraction for each state (hot and cold) based on the information held by the bettor.

    :param p_hot: Probability of winning when the deck is hot
    :param p_cold: Probability of winning when the deck is cold
    :param x1_win: Payoff when the deck is hot and the bettor wins (double the amount invested)
    :param x1_lose: Payoff when the deck is hot and the bettor loses (completely lose the amount invested)
    :param x2_win: Payoff when the deck is cold and the bettor wins (double the amount invested)
    :param x2_lose: Payoff when the deck is cold and the bettor loses (completely lose the amount invested)
    :return: Tuple containing the optimal betting fraction for hot and cold states
    """
    optimal_fraction_hot = find_optimal_fraction(p_hot, x1_win, x1_lose)
    optimal_fraction_cold = find_optimal_fraction(p_cold, x2_win, x2_lose)
    return (optimal_fraction_hot, optimal_fraction_cold)

In [7]:
# Example usage
p_hot = 0.52
p_cold = 0.48
x1_win = 2  # Double the amount invested when the deck is hot and the bettor wins
x1_lose = 0  # Completely lose the amount invested when the deck is hot and the bettor loses
x2_win = 2  # Double the amount invested when the deck is cold and the bettor wins
x2_lose = 0  # Completely lose the amount invested when the deck is cold and the bettor loses

optimal_fractions = find_optimal_fraction_hot_cold(p_hot, p_cold, x1_win, x1_lose, x2_win, x2_lose)
print(f"Optimal betting fraction when the deck is hot: {optimal_fractions[0]:.4f}")
print(f"Optimal betting fraction when the deck is cold: {optimal_fractions[1]:.4f}")

Optimal betting fraction when the deck is hot: 0.0400
Optimal betting fraction when the deck is cold: 0.0000


In [8]:
def find_optimal_fraction_hot_cold_constrained(p_hot, p_cold, x1_win, x1_lose, x2_win, x2_lose):
    """
    Find the optimal betting fraction for each state (hot and cold) with the constraint that the fraction bet
    when the deck is hot is three times larger than the fraction bet when the deck is cold.

    :param p_hot: Probability that the deck is hot
    :param p_cold: Probability that the deck is cold
    :param x1_win: Payoff when the deck is hot and the bettor wins (double the amount invested)
    :param x1_lose: Payoff when the deck is hot and the bettor loses (completely lose the amount invested)
    :param x2_win: Payoff when the deck is cold and the bettor wins (double the amount invested)
    :param x2_lose: Payoff when the deck is cold and the bettor loses (completely lose the amount invested)
    :return: Tuple containing the optimal betting fraction for hot and cold states
    """
    def objective_function(fractions):
        f_hot, f_cold = fractions
        return -(0.5 * expected_log_growth_rate(f_hot, p_hot, x1_win, x1_lose) +
                 0.5 * expected_log_growth_rate(f_cold, p_cold, x2_win, x2_lose))

    def constraint(fractions):
        f_hot, f_cold = fractions
        return f_hot - 3 * f_cold

    constraints = {'type': 'eq', 'fun': constraint}
    initial_guess = [0.1, 0.1]
    bounds = [(0, 1), (0, 1)]

    result = minimize(objective_function, initial_guess, bounds=bounds, constraints=constraints)

    return result.x

In [9]:
# Example usage
optimal_fractions_constrained = find_optimal_fraction_hot_cold_constrained(p_hot, p_cold, x1_win, x1_lose, x2_win, x2_lose)
print(f"Optimal betting fraction when the deck is hot (with constraint): {optimal_fractions_constrained[0]:.4f}")
print(f"Optimal betting fraction when the deck is cold (with constraint): {optimal_fractions_constrained[1]:.4f}")

Optimal betting fraction when the deck is hot (with constraint): 0.0241
Optimal betting fraction when the deck is cold (with constraint): 0.0080


In [10]:
def compare_growth_rates(optimal_fractions_unconstrained, optimal_fractions_constrained, p_hot, p_cold, x1_win, x1_lose, x2_win, x2_lose):
    """
    Compare the expected logarithmic growth rates for both the unconstrained and constrained cases.

    :param optimal_fractions_unconstrained: Tuple containing the optimal betting fraction for hot and cold states (unconstrained)
    :param optimal_fractions_constrained: Tuple containing the optimal betting fraction for hot and cold states (constrained)
    :param p_hot: Probability that the deck is hot
    :param p_cold: Probability that the deck is cold
    :param x1_win: Payoff when the deck is hot and the bettor wins (double the amount invested)
    :param x1_lose: Payoff when the deck is hot and the bettor loses (completely lose the amount invested)
    :param x2_win: Payoff when the deck is cold and the bettor wins (double the amount invested)
    :param x2_lose: Payoff when the deck is cold and the bettor loses (completely lose the amount invested)
    :return: None
    """
    f_hot_unconstrained, f_cold_unconstrained = optimal_fractions_unconstrained
    f_hot_constrained, f_cold_constrained = optimal_fractions_constrained

    growth_rate_unconstrained_hot = expected_log_growth_rate(f_hot_unconstrained, p_hot, x1_win, x1_lose) * 100
    growth_rate_unconstrained_cold = expected_log_growth_rate(f_cold_unconstrained, p_cold, x2_win, x2_lose) * 100
    growth_rate_unconstrained = 0.5 * (growth_rate_unconstrained_hot + growth_rate_unconstrained_cold)

    growth_rate_constrained_hot = expected_log_growth_rate(f_hot_constrained, p_hot, x1_win, x1_lose) * 100
    growth_rate_constrained_cold = expected_log_growth_rate(f_cold_constrained, p_cold, x2_win, x2_lose) * 100
    growth_rate_constrained = 0.5 * (growth_rate_constrained_hot + growth_rate_constrained_cold)

    print("Unconstrained case:")
    print(f"Optimal betting fractions - Hot: {f_hot_unconstrained * 100:.2f}%, Cold: {f_cold_unconstrained * 100:.2f}%")
    print(f"Expected logarithmic growth rate when the deck is hot: {growth_rate_unconstrained_hot:.2f}%")
    print(f"Expected logarithmic growth rate when the deck is cold: {growth_rate_unconstrained_cold:.2f}%")
    print(f"Overall expected logarithmic growth rate: {growth_rate_unconstrained:.2f}%\n")

    print("Constrained case:")
    print(f"Optimal betting fractions - Hot: {f_hot_constrained * 100:.2f}%, Cold: {f_cold_constrained * 100:.2f}%")
    print(f"Expected logarithmic growth rate when the deck is hot: {growth_rate_constrained_hot:.2f}%")
    print(f"Expected logarithmic growth rate when the deck is cold: {growth_rate_constrained_cold:.2f}%")
    print(f"Overall expected logarithmic growth rate: {growth_rate_constrained:.2f}%")

In [11]:
# Example
compare_growth_rates(optimal_fractions, optimal_fractions_constrained, p_hot, p_cold, x1_win, x1_lose, x2_win, x2_lose)

Unconstrained case:
Optimal betting fractions - Hot: 4.00%, Cold: 0.00%
Expected logarithmic growth rate when the deck is hot: 0.08%
Expected logarithmic growth rate when the deck is cold: -0.00%
Overall expected logarithmic growth rate: 0.04%

Constrained case:
Optimal betting fractions - Hot: 2.41%, Cold: 0.80%
Expected logarithmic growth rate when the deck is hot: 0.07%
Expected logarithmic growth rate when the deck is cold: -0.04%
Overall expected logarithmic growth rate: 0.02%


# Risk aversion on growth rates

In [12]:
p = 0.5  # Probability that the asset pays x1 for every dollar wagered
x1 = 3  # Payoff for every dollar wagered with probability p
x2 = .1  # Payoff for every dollar wagered with probability (1-p)


In [13]:
# Solve the example
optimal_fraction = find_optimal_fraction(p, x1, x2)
print(f"Optimal fraction of wealth to invest in the risky asset: {optimal_fraction:.4f}")
print(f"Optimal expected logarithmic growth rate: {expected_log_growth_rate(optimal_fraction, p, x1, x2):.4f}")

Optimal fraction of wealth to invest in the risky asset: 0.3056
Optimal expected logarithmic growth rate: 0.0777


In [14]:
# K> 1 is the multiple of w that makes the person risk losing everything
# 0 < K < 1 is the fraction of w the person protects in the losing state
K = .5
gamma = 1 - math.log2(K)**(-1)
print(gamma)

2.0


In [15]:
def expected_utility_crra(w_initial, p, x1, x2, q):
    """
    Compute the expected utility of a gamble for a CRRA person with RRA coefficient q.

    :param w_initial: Initial wealth
    :param p: Probability that the asset pays x1 for every dollar wagered
    :param x1: Payoff for every dollar wagered with probability p
    :param x2: Payoff for every dollar wagered with probability (1-p)
    :param q: RRA coefficient (q > 0)
    :return: Expected utility of the gamble
    """
    if q < 0:
        raise ValueError("RRA coefficient q must be non-negative")

    w1 = w_initial * x1
    w2 = w_initial * x2

    if q != 1:
        utility1 = (w1 ** (1 - q) - 1) / (1 - q)
        utility2 = (w2 ** (1 - q) - 1) / (1 - q)
    else:
        utility1 = np.log(w1)
        utility2 = np.log(w2)

    expected_utility = p * utility1 + (1 - p) * utility2
    return expected_utility

In [16]:
def find_optimal_fraction_crra(w_initial, p, x1, x2, q):
    """
    Find the optimal fraction of wealth to invest in a risky asset for a CRRA person with RRA coefficient q.

    :param w_initial: Initial wealth
    :param p: Probability that the asset pays x1 for every dollar wagered
    :param x1: Payoff for every dollar wagered with probability p
    :param x2: Payoff for every dollar wagered with probability (1-p)
    :param q: RRA coefficient (q > 0)
    :return: Optimal fraction of wealth to invest in the risky asset
    """
    def negative_expected_utility(fraction):
        x1_adjusted = 1 + fraction * (x1 - 1)
        x2_adjusted = 1 + fraction * (x2 - 1)
        return -expected_utility_crra(w_initial, p, x1_adjusted, x2_adjusted, q)

    result = minimize_scalar(negative_expected_utility, bounds=(0, 1), method='bounded')
    optimal_fraction = result.x
    return optimal_fraction


In [17]:
# Solve the example
K = .5
gamma = 1 - math.log2(K)**(-1)
optimal_fraction = find_optimal_fraction_crra(1, p, x1, x2,gamma)
print(f"Optimal fraction of wealth to invest in the risky asset: {optimal_fraction:.4f}")
print(f"Optimal expected logarithmic growth rate: {expected_log_growth_rate(optimal_fraction, p, x1, x2):.4f}")

Optimal fraction of wealth to invest in the risky asset: 0.1468
Optimal expected logarithmic growth rate: 0.0579
